In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [1]:
# from langchain.llms.openai import OpenAI
# from langchain.chat_models import ChatOpenAI, ChatAnthropic

# llm = OpenAI()
# chat = ChatOpenAI()

In [5]:
# AI에 일종의 기본 설정, 기본 값, 기본 context를 설정한 것
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [9]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
# p = CommaOutputParser()
# p.parse("Hello, how, are, you")

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a list generating machin. Everything you are asked will be answerd with a comma separated list of max {max_items} in lower case. Do NOT replay with anything else.'),
    ('human', '{question}'),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [14]:
chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a list generating machin. Everything you are asked will be answerd with a comma separated list of max {max_items} in lower case. Do NOT replay with anything else.'),
    ('human', '{question}'),
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 10,
    "question": "What are the pokemons?"
})


['pikachu',
 'charizard',
 'bulbasaur',
 'squirtle',
 'jigglypuff',
 'eevee',
 'snorlax',
 'mewtwo',
 'articuno',
 'zapdos']

In [15]:
# chain_one = template | chat | CommaOutputParser()
# chain_two = template_2 | chat | OntherOutputParser
# all = chain_one | chain_two

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])


chef_chain = chef_prompt | chat


In [23]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian",
})

Great choice! Indian cuisine is known for its vibrant flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy-to-follow recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pa

AIMessageChunk(content="Great choice! Butter Chicken is a delicious and popular Indian dish. To make it vegetarian, we can replace the chicken with a plant-based alternative. Here's an alternative recipe for Vegetarian Butter Chicken:\n\nIngredients:\n- 500g plant-based chicken substitute (such as tofu, tempeh, or seitan), cut into bite-sized pieces\n- 2 tablespoons butter or vegan butter substitute\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream or vegan heavy cream substitute\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix 